In [1]:
# instalar dependencias
%pip install transformers
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# clasificación de texto
# modelo preentrenado de OpenAI para clasificar reseñas de clientes en positivas o negativas:

from transformers import pipeline

classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
result = classifier("Este producto es increíble, me encantó.")
print(result)


c:\Users\jdmartinev\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jdmartinev\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\jdmartinev\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jdmartinev\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still wor

[{'label': '5 stars', 'score': 0.9009671807289124}]


In [3]:
# analisis de sentimientos

from transformers import pipeline
analyzer = pipeline("sentiment-analysis")
analyzer("No me gustó el servicio, fue una experiencia terrible.")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\jdmartinev\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\jdmartinev\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jdmartinev\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Cach

[{'label': 'NEGATIVE', 'score': 0.9965198040008545}]

In [4]:
# Ejemplo 1 con BERT-QA

from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

context = "El modelo de lenguaje BERT fue desarrollado por Google AI en 2018."
question = "¿Quién desarrolló BERT?"

result = qa_pipeline(question=question, context=context)
print(result['answer'])


c:\Users\jdmartinev\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jdmartinev\.cache\huggingface\hub\models--deepset--bert-base-cased-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at deepset/bert-base-cased-squad

modelo de lenguaje


In [6]:
result

{'score': 0.03815583884716034,
 'start': 3,
 'end': 21,
 'answer': 'modelo de lenguaje'}

In [7]:
# ejemplo 2 - Uso de GPT-4 para Q&A Generativo
# actualización: https://github.com/openai/openai-python
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "¿Quién descubrió la gravedad?",
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content)



OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# Ejemplo 3: Q&A Mejorado con Recuperación de Información (RAG)
# se verá más adelante!!!!

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from openai import OpenAI

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

# Cargar la base de datos de documentos en ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

# Crear un retriever para buscar información en la base de datos
retriever = vectorstore.as_retriever()

# Crear la Conversational Retrieval Chain con GPT-4
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4"),
    retriever=retriever
)

# Hacer una pregunta con recuperación de documentos
query = "¿Qué es LangChain?"
response = qa_chain({"question": query})
print(response["answer"])


In [ ]:
# summarization - resumen - Ejemplo con T5

from transformers import pipeline
summarizer = pipeline("summarization")

text = "Los modelos de lenguaje han cambiado la forma en que interactuamos con la tecnología..."
summary = summarizer(text, max_length=50, min_length=20, do_sample=False)

print(summary[0]['summary_text'])


In [ ]:
# Generación de texto - Ejemplo con GPT-4

import os
from openai import OpenAI

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Escribe un poema sobre la inteligencia artificial.",
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content)



In [ ]:
# Chatbots y Asistentes Virtuales - Ejemplo con LangChain y GPT-4

from langchain.chat_models import ChatOpenAI

chatbot = ChatOpenAI(model_name="gpt-4")
response = chatbot.predict("¿Cuáles son los beneficios de la IA?")
print(response)


In [ ]:
# Ejemplo 3: Q&A Mejorado con Recuperación de Información (RAG)
# Uso de LangChain con ChromaDB
#
import openai
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)
# Cargar la base de datos de documentos en ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

# Crear un retriever para buscar información en la base de datos
retriever = vectorstore.as_retriever()

# Crear la Conversational Retrieval Chain con GPT-4
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4"),
    retriever=retriever
)

# Hacer una pregunta con recuperación de documentos
query = "¿Qué es LangChain?"
response = qa_chain({"question": query})
print(response["answer"])
